In [1]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
import bff
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

Autosaving every 30 seconds


In [2]:
#import main df
df = pd.read_excel("data63.xlsx")

#load predictor dataframe
predictordata = pd.read_excel("PredictorData.xlsx")

#merge main dataframe with predictor data & wvs survey data
df = df.merge(predictordata, on='Country', how='left')

In [3]:
df = df.rename(columns={"IndividualismHI": "Ind_score"})
#create variable split around 50
df['Ind_ms'] = df['Ind_score'].apply(lambda x: 1 if x < 50 else 2)

#global median split
df_col = df.loc[df['Ind_ms']==1] #dataframe for collectivist countries
df_ind = df.loc[df['Ind_ms']==2] #dataframe for individualist countries
df = df.dropna(subset=['Ind_ms'])
#59440 participants

## Supplemental Analyses

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

%R library(lme4)
%R library(lmerTest)
%R library(ordinal)

Loading required package: Matrix



Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step



array(['ordinal', 'lmerTest', 'lme4', 'Matrix', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

# 1.1 Belief

In [5]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "EPI", 'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "EPI"],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'aControl')
df_belief = df_belief.dropna(subset=["ResponseId", "Belief", "condName"])

In [6]:
#create separate dataframes
df_col_belief = df_belief.loc[df_belief['Ind_ms']==1] #dataframe for collectivist countries
df_ind_belief = df_belief.loc[df_belief['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_belief
%Rpush df_ind_belief

### Collectivist Countries

In [7]:
%%R
M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_col_belief)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_col_belief.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_col_belief

REML criterion at convergence: 819139.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.0604 -0.2144  0.1154  0.2876  5.4822 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 337.7076 18.3768 
 Country    (Intercept)  30.1425  5.4902 
 Item       (Intercept)   0.3442  0.5866 
 Residual               162.2155 12.7364 
Number of obs: 96517, groups:  ResponseId, 24187; Country, 38; Item, 4

Fixed effects:
                          Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)              8.259e+01  1.033e+00 5.523e+01  79.951  < 2e-16 ***
condNameBindingMoral     5.196e-01  6.054e-01 2.407e+04   0.858 0.390776    
condNameCollectAction    1.803e+00  6.006e-01 2.407e+04   3.002 0.002684 ** 
condNameDynamicNorm      5.408e-01  6.011e-01 2.408

### Individualist Countries

In [8]:
%%R
M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_ind_belief)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_ind_belief.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_ind_belief

REML criterion at convergence: 1175136

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4546 -0.2923  0.0807  0.3003  6.2571 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 612.6244 24.7513 
 Country    (Intercept)  27.5571  5.2495 
 Item       (Intercept)   0.2904  0.5389 
 Residual               138.2471 11.7579 
Number of obs: 138245, groups:  ResponseId, 34595; Country, 25; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 76.1891     1.1852    35.5985  64.282  < 2e-16 ***
condNameBindingMoral         1.5327     0.6617 34552.3131   2.316  0.02055 *  
condNameCollectAction        2.7016     0.6610 34549.6414   4.087 4.37e-05 ***
condNameDynamicNorm          0.8712     0.6

# 1.2 Policy Support

In [9]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "EPI",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "EPI"],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'aControl')
df_policy = df_policy.dropna(subset=["ResponseId", "Policy", "condName"])

In [10]:
df_col_policy = df_policy.loc[df_policy['Ind_ms']==1] #dataframe for collectivist countries
df_ind_policy = df_policy.loc[df_policy['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_policy
%Rpush df_ind_policy

### Collectivist Countries

In [11]:
%%R
M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_col_policy)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_col_policy.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_col_policy

REML criterion at convergence: 1901984

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6188 -0.4948  0.1103  0.6516  4.3458 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 220.42   14.847  
 Country    (Intercept)  22.03    4.693  
 Item       (Intercept) 125.60   11.207  
 Residual               449.52   21.202  
Number of obs: 208145, groups:  ResponseId, 24037; Country, 38; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 72.6084     3.8303     8.8396  18.956 1.83e-08 ***
condNameBindingMoral         0.5359     0.5159 23797.5258   1.039   0.2990    
condNameCollectAction        2.4242     0.5116 23787.4935   4.739 2.16e-06 ***
condNameDynamicNorm          1.0544     0.5

### Individualist Countries

In [12]:
%%R
M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_ind_policy)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_ind_policy.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_ind_policy

REML criterion at convergence: 2742990

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8523 -0.5431  0.0448  0.6275  4.8557 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 365.22   19.111  
 Country    (Intercept)  14.38    3.792  
 Item       (Intercept) 153.63   12.395  
 Residual               413.26   20.329  
Number of obs: 301125, groups:  ResponseId, 34345; Country, 25; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 66.9462     4.2182     8.6787  15.871 1.04e-07 ***
condNameBindingMoral         0.8471     0.5308 34226.3608   1.596  0.11051    
condNameCollectAction        3.2704     0.5307 34217.3241   6.163 7.23e-10 ***
condNameDynamicNorm          1.1913     0.5

# 1.3 Sharing

In [13]:
cols_to_check = ["ResponseId", "condName", "SHAREcc"]
df_share = df.dropna(subset=cols_to_check)
df_share['condName'] = df_share['condName'].replace(['Control'], 'aControl')

In [14]:
df_col_share = df_share.loc[df_share['Ind_ms']==1] #dataframe for collectivist countries
df_ind_share = df_share.loc[df_share['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_share
%Rpush df_ind_share

### Collectivist Countries

In [15]:
%%R
M1 <- glmer(SHAREcc ~ condName + (1 | Country), family = binomial, data = df_col_share)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_col_share.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_col_share

     AIC      BIC   logLik deviance df.resid 
 20390.7  20492.9 -10182.4  20364.7    19115 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3089 -0.6039  0.4191  0.5981  2.5610 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.206    1.098   
Number of obs: 19128, groups:  Country, 38

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)               0.42912    0.18710   2.294 0.021818 *  
condNameBindingMoral      0.30532    0.08075   3.781 0.000156 ***
condNameCollectAction     0.60396    0.08247   7.323 2.43e-13 ***
condNameDynamicNorm       0.45771    0.08129   5.631 1.80e-08 ***
condNameFutureSelfCont    0.41963    0.08591   4.884 1.04e-06 ***
condNameLetterFutureGen   0.42237    0.08657   4.879 1.07e-06 ***


### Individualist Countries

In [16]:
%%R
M1 <- glmer(SHAREcc ~ condName + (1 | Country), family = binomial, data = df_ind_share)
print(summary(M1))

write.csv(coef(summary(M1)), "coef_ind_share.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_ind_share

     AIC      BIC   logLik deviance df.resid 
 31354.9  31460.3 -15664.4  31328.9    24515 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9760 -0.8137 -0.4808  0.8616  2.4138 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.5497   0.7414  
Number of obs: 24528, groups:  Country, 25

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -0.54519    0.15591  -3.497 0.000471 ***
condNameBindingMoral      0.11517    0.06472   1.780 0.075127 .  
condNameCollectAction     0.26636    0.06567   4.056 5.00e-05 ***
condNameDynamicNorm       0.21922    0.06514   3.365 0.000764 ***
condNameFutureSelfCont    0.25089    0.06889   3.642 0.000271 ***
condNameLetterFutureGen   0.37937    0.07037   5.391 6.99e-08 ***


# 1.4 WEPT

In [17]:
cols_to_check = ["ResponseId", "condName", "WEPTcc"]
df_WEPT = df.dropna(subset=cols_to_check)
df_WEPT['condName'] = df_WEPT['condName'].replace(['Control'], 'aControl')

In [18]:
df_col_WEPT = df_WEPT.loc[df_WEPT['Ind_ms']==1] #dataframe for collectivist countries
df_ind_WEPT = df_WEPT.loc[df_WEPT['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_WEPT
%Rpush df_ind_WEPT

### Collectivist Countries

In [19]:
%%R
M1 <- clmm(as.factor(WEPTcc) ~ condName + (1 | Country), data = df_col_WEPT, threshold = "equidistant")
print(summary(M1))

write.csv(coef(summary(M1)), "coef_col_wept.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_col_WEPT

 link  threshold   nobs  logLik    AIC      niter      max.grad cond.H 
 logit equidistant 24233 -39844.78 79717.55 1967(7873) 7.79e-02 5.7e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2804   0.5295  
Number of groups:  Country 38 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral     -0.04643    0.05969  -0.778  0.43661    
condNameCollectAction    -0.10226    0.05900  -1.733  0.08303 .  
condNameDynamicNorm      -0.04161    0.05919  -0.703  0.48198    
condNameFutureSelfCont   -0.17535    0.06169  -2.843  0.00448 ** 
condNameLetterFutureGen  -0.24773    0.06136  -4.037 5.41e-05 ***
condNameNegativeEmotions -0.32771    0.05848  -5.604 2.09e-08 ***
condNamePluralIgnorance  -0.13633    0.05948  -2.292  0.02190 *  
condNamePsychDistance    -0.33341    0.06101

### Individualist Countries

In [20]:
%%R
M1 <- clmm(as.factor(WEPTcc) ~ condName + (1 | Country), data = df_ind_WEPT, threshold = "equidistant")
print(summary(M1))

write.csv(coef(summary(M1)), "coef_ind_wept.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_ind_WEPT

 link  threshold   nobs  logLik    AIC       niter      max.grad cond.H 
 logit equidistant 34647 -56474.87 112977.75 1664(4997) 1.04e-01 8.2e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1401   0.3743  
Number of groups:  Country 25 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral      0.07716    0.04960   1.555  0.11983    
condNameCollectAction    -0.12802    0.04895  -2.615  0.00891 ** 
condNameDynamicNorm       0.03135    0.04928   0.636  0.52464    
condNameFutureSelfCont   -0.08601    0.05206  -1.652  0.09847 .  
condNameLetterFutureGen  -0.21411    0.05254  -4.075 4.60e-05 ***
condNameNegativeEmotions -0.26765    0.04856  -5.511 3.56e-08 ***
condNamePluralIgnorance  -0.02130    0.04899  -0.435  0.66378    
condNamePsychDistance    -0.23897    0.049